# Day Night Image Classifier

In [2]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import helpers

In [ ]:
# Image data directories
image_dir_training = "day_night_images/training/"
image_dir_test = "day_night_images/test/"


# Load training data
IMAGE_LIST = helpers.load_dataset(image_dir_training)


# Select an image and its label by list index
image_index = 0
selected_image = IMAGE_LIST[image_index][0]
selected_label = IMAGE_LIST[image_index][1]

### Standardizing

In [ ]:
#standardizing input
def standardize_input(image):
    #Resize image so that all "standard" images are the same size 600x1100 (hxw) 
    standard_im = image[:600,:1100]
    
    return standard_im

#standardizing input
def encode(label):
    numerical_val = 0
    #labelling data
    if (label=="day"):
        numerical_val=1
    else:
        numerical_val=0
        
    return numerical_val

#constructing standardized list
def standardize(image_list):
    standard_list = []
    
    for item in image_list:
        image = item[0]
        label = item[1]
        
        standardized_im = standardize_input(image)
        binary_label = encode(label)    
        standard_list.append((standardized_im, binary_label))
        
    return standard_list



STANDARDIZED_LIST = helpers.standardize(IMAGE_LIST)

# Selecting an image by index
image_num = 0
selected_image = STANDARDIZED_LIST[image_num][0]
selected_label = STANDARDIZED_LIST[image_num][1]

### Average brightness using V channel in HSV

In [ ]:
def avg_brightness(rgb_image):
    # Convert image to HSV
    hsv = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2HSV)
    # Add up all the pixel values in the V channel
    sum_brightness = np.sum(hsv[:,:,2])
    area = 600*1100.0  # pixels
    avg = sum_brightness/area
    
    return avg

### predict label

In [ ]:
def estimate_label(rgb_image):
    avg = avg_brightness(rgb_image)
    predicted_label = 0
    threshold = 105
    if(avg > threshold):
        predicted_label = 1
    
    return predicted_label

### Determining Accuracy

In [ ]:
import random

TEST_IMAGE_LIST = helpers.load_dataset(image_dir_test)
STANDARDIZED_TEST_LIST = helpers.standardize(TEST_IMAGE_LIST)

# Shuffle the standardized test data
random.shuffle(STANDARDIZED_TEST_LIST)


def get_misclassified_images(test_images):
    misclassified_images_labels = []
    
    # Classify each image and compare to the true label
    for image in test_images:
        im = image[0]
        true_label = image[1]
        predicted_label = estimate_label(im)

        if(predicted_label != true_label):
            misclassified_images_labels.append((im, predicted_label, true_label))
            
    return misclassified_images_labels


#Finding all misclassified images in a given test set
MISCLASSIFIED = get_misclassified_images(STANDARDIZED_TEST_LIST)

# Accuracy calculations
total = len(STANDARDIZED_TEST_LIST)
num_correct = total - len(MISCLASSIFIED)
accuracy = num_correct/total

print('Accuracy: ' + str(accuracy))
print("Number of misclassified images = " + str(len(MISCLASSIFIED)) +' out of '+ str(total))


# Visualize misclassified example(s)
num = 0
test_mis_im = MISCLASSIFIED[num][0]
predict_label = MISCLASSIFIED[num][1] 
true_label = MISCLASSIFIED[num][2] 
plt.imshow(test_mis_im)
print(predict_label)
print(true_label)